In [1]:
Path(constants["DATA_PATH"]) / "encodings" / "encodings_whole_song.pkl"%cd ..

/home/chris/DSR/plum_music


In [3]:
from pathlib import Path
from pprint import pprint

import models.preprocessing as pre
from utils.data_utils import make_metadata_csv
from utils.lakh_utils import *
from utils.magenta_midi_utils import *
from utils.magenta_models_utils import *
from utils.magenta_note_seq_utils import *
from config.config import get_constants_dict, set_up

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.manifold import TSNE

import note_seq as ns
import pretty_midi
from magenta.models import music_vae
from magenta.models.music_vae.trained_model import NoExtractedExamplesError, MultipleExtractedExamplesError
from mido import MidiFile

## Get sample tracks

### Load metadata and configs

In [4]:
# does not check if the file already exist!!
# so make sure to only use it if you do not have a current version of it
# make_metadata_csv()

In [5]:
set_up()
const = get_constants_dict()

In [6]:
metadata = pd.read_csv(Path(const["LMD_METADATA_CSV_FILE"]))
matches = get_msd_score_matches()

In [25]:
genres = metadata.groupby("mb_genre").nunique()
genres[genres.msdID >= 90]

,msdID,md5,artist,track,album,mb_genre_count
mb_genre,,,,,,
alternative rock,222,185,32,177,119,4
american,503,422,141,426,370,4
australian,101,88,23,93,69,3
british,835,682,148,701,544,5
canadian,184,156,35,152,108,4
classic pop and rock,1396,1063,371,1056,997,1
classical,369,330,104,358,216,4
country,344,293,77,293,224,4
dance and electronica,95,86,58,88,79,1




### Define sample tracks for three genres (country, metal, pop)

Dolly Parton - Jolene

In [6]:
# Dolly Parton - Jolene
country_msd_id = "TRCFSXM12903CC84F4"
country_md5 = get_matched_midi_md5(country_msd_id, matches)
country_midi_path = get_midi_path(country_msd_id, country_md5)
metadata[metadata.msdID == country_msd_id]

,msdID,md5,artist,track,album,mb_genre,mb_genre_count
16798,TRCFSXM12903CC84F4,f3cc01515b7dfe1a2bb1e36c73621309,Dolly Parton,Jolene,Essential - Country,country,2.0


In Flames - Only For The Weak

In [7]:
# In Flames - Only For The Weak
metal_msd_id = "TRSIRKX128F4291CAF"
metal_md5 = get_matched_midi_md5(metal_msd_id, matches)
metal_midi_path = get_midi_path(metal_msd_id, metal_md5)
metadata[metadata.msdID == metal_msd_id]

,msdID,md5,artist,track,album,mb_genre,mb_genre_count
17104,TRSIRKX128F4291CAF,f7f9778ce935e55b10a912503becadc3,In Flames,Only For The Weak,The Tokyo Showdown,metal,5.0


Britney Spears - ...Baby One More Time

In [8]:
metadata[metadata.artist.str.contains("Britney") & metadata.track.str.contains("Time")]

,msdID,md5,artist,track,album,mb_genre,mb_genre_count
12088,TREUBTU12903CC84A6,b9e4cddf5f58c369c217121da91d40b8,Britney Spears,...Baby One More Time,Essential - Girls Night In,pop,6.0
18948,TRBKMAL128F92D5092,b9e4cddf5f58c369c217121da91d40b8,Britney Spears,...Baby One More Time,Girls Night In,pop,6.0


In [9]:
# Britney Spears - Oops! I did it again...
pop_msd_id = "TREUBTU12903CC84A6"
pop_md5 = get_matched_midi_md5(pop_msd_id, matches)
pop_midi_path = get_midi_path(pop_msd_id, pop_md5)
metadata[metadata.msdID == pop_msd_id]

,msdID,md5,artist,track,album,mb_genre,mb_genre_count
12088,TREUBTU12903CC84A6,b9e4cddf5f58c369c217121da91d40b8,Britney Spears,...Baby One More Time,Essential - Girls Night In,pop,6.0


### Prepare the midi files (create new split and cleaned files)

In [10]:
new_midis = pre.prepare_midi([country_msd_id, metal_msd_id, pop_msd_id])
print(new_midis)

[PosixPath('/home/chris/DSR/plum_music/data/cleaned/f3cc01515b7dfe1a2bb1e36c73621309'), PosixPath('/home/chris/DSR/plum_music/data/cleaned/f7f9778ce935e55b10a912503becadc3'), PosixPath('/home/chris/DSR/plum_music/data/cleaned/b9e4cddf5f58c369c217121da91d40b8')]


### Get the directories of the newly created files

Get lists of the cleaned files directories for each genre

In [11]:
country_cleaned = new_midis[0]
metal_cleaned = new_midis[1]
pop_cleaned = new_midis[2]

### Get the cleaned files with all tracks

In [48]:
country_note_seq = ns.midi_file_to_note_sequence(
    next(new_midis[0].glob("*cleaned.mid"))
)
metal_note_seq = ns.midi_file_to_note_sequence(
    next(new_midis[1].glob("*cleaned.mid"))
)
pop_note_seq = ns.midi_file_to_note_sequence(
    next(new_midis[2].glob("*cleaned.mid"))
)

## Split sequences into bars of model length input

### Check if only one tempo is defined for each sequence

In [13]:
pop_single_tempo = ns.sequences_lib.split_note_sequence_on_time_changes(pop_note_seq)
country_single_tempo = ns.sequences_lib.split_note_sequence_on_time_changes(country_note_seq)
metal_single_tempo = ns.sequences_lib.split_note_sequence_on_time_changes(metal_note_seq)

In [14]:
pop_single_tempo = [n for n in pop_single_tempo if n.total_time > 1.]
country_single_tempo = [n for n in country_single_tempo if n.total_time > 1.]
metal_single_tempo = [n for n in metal_single_tempo if n.total_time > 1.]
print(
    f"Num segments after split on tempo change:\n"
    f"\tPop: {len(pop_single_tempo)}"
    f"\tCountry: {len(country_single_tempo)}"
    f"\tMetal: {len(metal_single_tempo)}"
)

Num segments after split on tempo change:
	Pop: 1	Country: 1	Metal: 1


### Split into n bars

In [15]:
n_bars = 1
country_1bar_splits = pre.split_sequence_by_num_bars(country_note_seq, n_bars)
metal_1bar_splits = pre.split_sequence_by_num_bars(metal_note_seq, n_bars)
pop_1bar_splits = pre.split_sequence_by_num_bars(pop_note_seq, n_bars)

In [16]:
n_bars = 16
country_16bar_splits = pre.split_sequence_by_num_bars(country_note_seq, n_bars)
metal_16bar_splits = pre.split_sequence_by_num_bars(metal_note_seq, n_bars)
pop_16bar_splits = pre.split_sequence_by_num_bars(pop_note_seq, n_bars)

## Do fancy model stuff

### Hierdec_Mel_16_bars

In [17]:
model_config = music_vae.configs.CONFIG_MAP[const["CHECKPOINT_MUSICVAE_HIERDEC_MEL"]]

In [18]:
model = get_model(const["CHECKPOINT_MUSICVAE_HIERDEC_MEL"])

--- Logging error ---
Traceback (most recent call last):
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 611, in format
    s = self.formatMessage(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 580, in formatMessage
    return self._style.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 422, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_g

--- Logging error ---
Traceback (most recent call last):
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 611, in format
    s = self.formatMessage(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 580, in formatMessage
    return self._style.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 422, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_g

--- Logging error ---
Traceback (most recent call last):
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 611, in format
    s = self.formatMessage(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 580, in formatMessage
    return self._style.format(record)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/logging/__init__.py", line 422, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/chris/anaconda3/envs/project/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_g

#### Get the tracks

In [19]:
country_tracks_paths = sorted([track for track in country_cleaned.glob("*track*")])
metal_tracks_paths = sorted([track for track in metal_cleaned.glob("*track*")])
pop_tracks_paths = sorted([track for track in pop_cleaned.glob("*track*")])

In [20]:
country_tracks = [
    ns.midi_file_to_note_sequence(str(path))
    for path in country_tracks_paths
]
metal_tracks = [
    ns.midi_file_to_note_sequence(str(path))
    for path in metal_tracks_paths
]
pop_tracks = [
    ns.midi_file_to_note_sequence(str(path))
    for path in pop_tracks_paths
]

In [21]:
ct = model_config.data_converter.to_tensors(metal_tracks[1])
ct[1] == ct.outputs

True

In [22]:
extracted_16_mels = []

In [23]:
for seq in metal_tracks:
    extracted_16_mels.extend(
        model_config.data_converter.from_tensors(
            model_config.data_converter.to_tensors(seq).outputs))

encodings = []
for seq in extracted_16_mels:
    try:
        encodings.append(model.encode([seq])[0])
    except NoExtractedExamplesError as ex:
        print(
            f"Got {type(ex)} for {seq.instrument_infos} while encoding"
        )
        continue
    except MultipleExtractedExamplesError as ex:
        print(
            f"Got {type(ex)} for {seq.instrument_infos} while encoding"
        )
        continue

enc = np.vstack(encodings)
enc.shape

#### Visualize with tsne

z_embedded = TSNE().fit_transform(enc)

z_embedded

plt.scatter(z_embedded[:, 0], z_embedded[:, 1])

px.scatter(x=z_embedded[:, 0], y=z_embedded[:, 1])

### Multitrack

In [24]:
checkpoint = "hier-multiperf_vel_1bar_med"

In [25]:
multi_model = get_model(checkpoint)
multi_config = music_vae.configs.CONFIG_MAP[checkpoint]

### Try to change the model configs

dc = music_vae.data_hierarchical.MultiInstrumentPerformanceConverter(
        num_velocity_bins=8,
        hop_size_bars=1,
        max_num_instruments=32,
        max_events_per_instrument=128
)

multiperf_encoder = music_vae.lstm_models.HierarchicalLstmEncoder(
    music_vae.lstm_models.BidirectionalLstmEncoder,
    level_lengths=[128, 32])
multiperf_decoder = music_vae.lstm_models.HierarchicalLstmDecoder(
    music_vae.lstm_models.CategoricalLstmDecoder(),
    level_lengths=[32, 128],
    disable_autoregression=True)

hparams_map = music_vae.base_model.get_default_hparams().values()
hparams_map.update({
      'conditional': True,
      'dec_rnn_size': [4096],  # Decoder RNN: number of units per layer.
      'enc_rnn_size': [256],  # Encoder RNN: number of units per layer per dir.
      'dropout_keep_prob': 1.0,  # Probability all dropout keep.
      'sampling_schedule': 'constant',  # constant, exponential, inverse_sigmoid
      'sampling_rate': 0.0,  # Interpretation is based on `sampling_schedule`.
      'use_cudnn': False,  # DEPRECATED
      'residual_encoder': False,  # Use residual connections in encoder.
      'residual_decoder': False,  # Use residual connections in decoder.
      'control_preprocessing_rnn_size': [256],  # Decoder control preprocessing.
  })
base_hparams = music_vae.lstm_models.contrib_training.HParams(**hparams_map)

multiperf_hparams_med = music_vae.configs.merge_hparams(
    base_hparams,
    music_vae.configs.HParams(
        batch_size=256,
        max_seq_len=4096,
        z_size=512,
        enc_rnn_size=[1024],
        dec_rnn_size=[512, 512, 512]))

new_conf = music_vae.configs.Config(
    model=music_vae.configs.MusicVAE(multiperf_encoder, multiperf_decoder),
    hparams=multiperf_hparams_med,
    note_sequence_augmenter=music_vae.data.NoteSequenceAugmenter(
        transpose_range=(-3, 3)),
    data_converter=dc,
    train_examples_path=None,
    eval_examples_path=None
)

checkpoint = Path(constants["MODELS_MUSICVAE_PATH"], "multitrack", "conditioned", constants["CHECKPOINT_MUSICVAE_MULTITRACK_CONDITIONED"])
checkpoint

m = TrainedModel(
        config=new_conf,
        batch_size=8,
        checkpoint_dir_or_path=str(checkpoint)
)

### Try to encode one sequence

## Testing area

In [26]:
pm = ns.sequence_proto_to_pretty_midi(metal_note_seq)

In [27]:
metal_instruments = pre.split_by_instrument(country_note_seq)

In [28]:
metal_inst_splits = [
    pre.split_sequence_by_num_bars(instrument, n_bars=16) for instrument in metal_instruments
]

for i, inst in enumerate(metal_inst_splits):
    print(f"Instrument no {i}")
    for split in inst:
        print(
            f"\tSplit has {len(split.notes)} notes"
        )

Instrument no 0
	Split has 261 notes
	Split has 22 notes
Instrument no 1
	Split has 30 notes
	Split has 15 notes
Instrument no 2
	Split has 308 notes
	Split has 92 notes
Instrument no 3
	Split has 2269 notes
	Split has 664 notes
Instrument no 4
	Split has 914 notes
	Split has 204 notes
Instrument no 5
	Split has 105 notes
	Split has 55 notes
Instrument no 6
	Split has 44 notes
	Split has 4 notes
Instrument no 7
	Split has 1 notes
Instrument no 8
	Split has 248 notes
	Split has 60 notes
Instrument no 9
	Split has 124 notes
	Split has 30 notes
Instrument no 10
	Split has 124 notes
	Split has 30 notes
Instrument no 11
	Split has 496 notes
	Split has 120 notes
Instrument no 12
	Split has 14 notes
Instrument no 13
	Split has 992 notes
	Split has 240 notes


In [29]:
melodies = [
    pre.extract_melodies(inst, const["CHECKPOINT_MUSICVAE_HIERDEC_MEL"])
    for inst in metal_inst_splits
]

In [30]:
encodings = []
num_encoded, num_no_extracted, num_multiple_extracted = 0, 0, 0
for inst in melodies:
    try:
        z, _, _ = model.encode(inst)
        encodings.append(z)
    except NoExtractedExamplesError as ex:
        #print(f"No extracted examples in {inst}")
        num_no_extracted += 1
        continue
    except MultipleExtractedExamplesError as ex:
        #print(f"Multiple extracted examples in {inst}")
        num_multiple_extracted += 1
        continue
    except Exception as ex:
        print(f"Something went wrong. Exception:")
        print(ex)
        continue
    else:
        num_encoded += 1
        
print(
    f"Num succesfully encoded: {num_encoded}\n"
    f"Num ignored due to no extracted: {num_no_extracted}\n"
    f"Num ignored due to multiple extracted: {num_multiple_extracted}"
)

Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Something went wrong. Exception:
max() arg is an empty sequence
Num succesfully encoded: 3
Num ignored due to no extracted: 0
Num ignored due to multiple extracted: 0


Multitrack

In [31]:
splits_track = pre.extract_multitracks(country_note_seq)

In [32]:
len(splits_track)

2

In [33]:
splits = [
    pre.split_sequence_by_num_bars(s, n_bars=1)
    for s in splits_track
]

In [34]:
multi_model._config.data_converter.to_tensors

<bound method MultiInstrumentPerformanceConverter.to_tensors of <magenta.models.music_vae.data_hierarchical.MultiInstrumentPerformanceConverter object at 0x7f0bbc5c3b90>>

In [35]:
ns.play_sequence(splits[0][2])

In [36]:
uploaded_seqs = []
for s in splits:
    for seq in s:
        print(type(seq))
        tensors = multi_model._config.data_converter.to_tensors(seq)
        print(tensors)
        #uploaded_seqs.extend(multi_model._config.data_converter.from_tensors(tensors))

<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])
<class 'note_seq.protobuf.music_pb2.NoteSequence'>
ConverterTensors(inpu

In [37]:
uploaded_seqs

[]

In [50]:
encs = []
errors = 0
for s in splits[0]:
    try:
        encs.append(multi_model.encode([s])[0])
    except NoExtractedExamplesError as ex:
        errors += 1
        #print(ex)
        continue
        
print(f"Num errors: {errors}")

Num errors: 20


In [41]:
encs

[]

In [52]:
multi_config.data_converter.to_tensors(
    ns.midi_file_to_note_sequence(country_midi_path)
)

ConverterTensors(inputs=(array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]),), outputs=(array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]),), controls=(array([], shape=(1, 0), dtype=float64),), lengths=(array([7, 7, 9, 7, 8, 7, 7, 7], dtype=int32),))

In [61]:
z, _, _ = multi_model.encode([ns.midi_file_to_note_sequence(country_midi_path)])

In [78]:
z, _, _ = multi_model.encode([country_note_seq])

NoExtractedExamplesError: No examples extracted from NoteSequence: ticks_per_quarter: 120
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
  time: 0.0125
}
tempos {
  qpm: 120.0
}
notes {
  pitch: 73
  velocity: 87
  start_time: 11.7375
  end_time: 11.875
  program: 17
}
notes {
  pitch: 73
  velocity: 93
  start_time: 12.004166666666666
  end_time: 12.375
  program: 17
}
notes {
  pitch: 76
  velocity: 95
  start_time: 12.725
  end_time: 12.854166666666666
  program: 17
}
notes {
  pitch: 76
  velocity: 95
  start_time: 12.979166666666666
  end_time: 13.345833333333333
  program: 17
}
notes {
  pitch: 78
  velocity: 93
  start_time: 13.737499999999999
  end_time: 13.866666666666667
  program: 17
}
notes {
  pitch: 78
  velocity: 101
  start_time: 14.016666666666666
  end_time: 14.525
  program: 17
}
notes {
  pitch: 83
  velocity: 99
  start_time: 14.504166666666666
  end_time: 15.0125
  program: 17
}
notes {
  pitch: 85
  velocity: 100
  start_time: 15.0
  end_tim

In [71]:
type(z)

numpy.ndarray

In [77]:
foo = ns.midi_file_to_note_sequence(country_midi_path)

TypeError: expected str, bytes or os.PathLike object, not NoteSequence

In [81]:
foo.tempos

[qpm: 120.0
, time: 0.0125
qpm: 140.00014000014
, time: 3.4410680000000005
qpm: 109.99990833340973
]